In [1]:
import numpy as np
import pandas as pd

from keras.models import Sequential
from keras.layers import Dense
from keras.utils import to_categorical
from keras.callbacks import EarlyStopping

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

Using TensorFlow backend.


In [2]:
train = pd.read_csv('DATASET/train.csv')
test = pd.read_csv('DATASET/test.csv')

In [3]:
print ('The train data has {} rows and {} columns'.format(train.shape[0],train.shape[1]))
print ('The test data has {} rows and {} columns'.format(test.shape[0],test.shape[1]))

The train data has 12137810 rows and 10 columns
The test data has 3706907 rows and 9 columns


In [4]:
train.head()

,ID,datetime,siteid,offerid,category,merchant,countrycode,browserid,devid,click
0,IDsrk7SoW,2017-01-14 09:42:09,4709696.0,887235,17714,20301556,e,Firefox,NaN,0
1,IDmMSxHur,2017-01-18 17:50:53,5189467.0,178235,21407,9434818,b,Mozilla Firefox,Desktop,0
2,IDVLNN0Ut,2017-01-11 12:46:49,98480.0,518539,25085,2050923,a,Edge,NaN,0
3,ID32T6wwQ,2017-01-17 10:18:43,8896401.0,390352,40339,72089744,c,Firefox,Mobile,0
4,IDqUShzMg,2017-01-14 16:02:33,5635120.0,472937,12052,39507200,d,Mozilla Firefox,Desktop,0


In [5]:
# imputing missing values
train['siteid'].fillna(-999, inplace=True)
test['siteid'].fillna(-999, inplace=True)

train['browserid'].fillna("None",inplace=True)
test['browserid'].fillna("None", inplace=True)

train['devid'].fillna("None",inplace=True)
test['devid'].fillna("None",inplace=True)

In [6]:

# create timebased features

train['datetime'] = pd.to_datetime(train['datetime'])
test['datetime'] = pd.to_datetime(test['datetime'])

train['tweekday'] = train['datetime'].dt.weekday
test['tweekday'] = test['datetime'].dt.weekday

train['tmonth'] = train['datetime'].dt.month
test['tmonth'] = test['datetime'].dt.month

train['tday'] = train['datetime'].dt.day
test['tday'] = test['datetime'].dt.day

train['tyear'] = train['datetime'].dt.year
test['tyear'] = test['datetime'].dt.year

train['thour'] = train['datetime'].dt.hour
test['thour'] = test['datetime'].dt.hour

train['tminute'] = train['datetime'].dt.minute
test['tminute'] = test['datetime'].dt.minute

In [7]:
# create aggregate features
site_offer_count = train.groupby(['siteid','offerid']).size().reset_index()
site_offer_count.columns = ['siteid','offerid','site_offer_count']

site_offer_count_test = test.groupby(['siteid','offerid']).size().reset_index()
site_offer_count_test.columns = ['siteid','offerid','site_offer_count']

site_cat_count = train.groupby(['siteid','category']).size().reset_index()
site_cat_count.columns = ['siteid','category','site_cat_count']

site_cat_count_test = test.groupby(['siteid','category']).size().reset_index()
site_cat_count_test.columns = ['siteid','category','site_cat_count']

site_mcht_count = train.groupby(['siteid','merchant']).size().reset_index()
site_mcht_count.columns = ['siteid','merchant','site_mcht_count']

site_mcht_count_test = test.groupby(['siteid','merchant']).size().reset_index()
site_mcht_count_test.columns = ['siteid','merchant','site_mcht_count']

In [8]:

# joining all files
agg_df = [site_offer_count,site_cat_count,site_mcht_count]
agg_df_test = [site_offer_count_test,site_cat_count_test,site_mcht_count_test]

for x in agg_df:
    train = train.merge(x)
    
for x in agg_df_test:
    test = test.merge(x)

In [9]:

# Label Encoding
from sklearn.preprocessing import LabelEncoder
for c in list(train.select_dtypes(include=['object']).columns):
    if c != 'ID':
        lbl = LabelEncoder()
        lbl.fit(list(train[c].values) + list(test[c].values))
        train[c] = lbl.transform(list(train[c].values))
        test[c] = lbl.transform(list(test[c].values))

In [10]:

# sample 10% data - to avoid memory troubles
# if you have access to large machines, you can use more data for training

train = train.sample(train.shape[0])
print (train.shape)

(12137810, 19)


In [11]:
# select columns to choose
cols_to_use = [x for x in train.columns if x not in list(['ID','datetime','click'])]

In [12]:

# standarise data before training
scaler = StandardScaler().fit(train[cols_to_use])

strain = scaler.transform(train[cols_to_use])
stest = scaler.transform(test[cols_to_use])

In [14]:
# train validation split
X_train, X_valid, Y_train, Y_valid = train_test_split(strain, train.click, test_size = 0.1, random_state=2017)

In [15]:
print (X_train.shape)
print (X_valid.shape)
print (Y_train.shape)
print (Y_valid.shape)

(10924029, 16)
(1213781, 16)
(10924029,)
(1213781,)


In [16]:
# model architechture
def keras_model(train):
    
    input_dim = train.shape[1]
    classes = 2
    
    model = Sequential()
    model.add(Dense(512, activation = 'relu', input_shape = (input_dim,)))
    model.add(Dense(256, activation = 'relu'))
    model.add(Dense(128, activation = 'relu'))
    model.add(Dense(64, activation = 'relu'))
    model.add(Dense(32, activation = 'relu'))
    model.add(Dense(classes, activation = 'sigmoid'))
    model.compile(optimizer = 'adam', loss='binary_crossentropy',metrics = ['accuracy'])
    return model

callback = EarlyStopping(monitor='val_acc',patience=3)

In [17]:
# one hot target columns
Y_train = to_categorical(Y_train)
Y_valid = to_categorical(Y_valid)

In [18]:
# train model
model = keras_model(X_train)
model.fit(X_train, Y_train, 2000, 50, callbacks=[callback],validation_data=(X_valid, Y_valid),shuffle=True)

Train on 10924029 samples, validate on 1213781 samples
Epoch 1/50
10924029/10924029 [==============================] - 38s - loss: 0.0673 - acc: 0.9772 - val_loss: 0.0620 - val_acc: 0.9785
Epoch 2/50
10924029/10924029 [==============================] - 38s - loss: 0.0594 - acc: 0.9797 - val_loss: 0.0577 - val_acc: 0.9803
Epoch 3/50
10924029/10924029 [==============================] - 37s - loss: 0.0564 - acc: 0.9807 - val_loss: 0.0552 - val_acc: 0.9810
Epoch 4/50
10924029/10924029 [==============================] - 37s - loss: 0.0548 - acc: 0.9812 - val_loss: 0.0550 - val_acc: 0.9811
Epoch 5/50
10924029/10924029 [==============================] - 37s - loss: 0.0538 - acc: 0.9815 - val_loss: 0.0574 - val_acc: 0.9804
Epoch 6/50
10924029/10924029 [==============================] - 37s - loss: 0.0533 - acc: 0.9817 - val_loss: 0.0535 - val_acc: 0.9818
Epoch 7/50
10924029/10924029 [==============================] - 38s - loss: 0.0526 - acc: 0.9819 - val_loss: 0.0553 - val_acc: 0.9812
Epoch 8

In [19]:
# check validation accuracy
vpreds = model.predict_proba(X_valid)[:,1]
from sklearn.metrics import roc_auc_score
roc_auc_score(y_true = Y_valid[:,1], y_score=vpreds)

1211808/1213781 [============================>.] - ETA: 0s

0.98102068719767144

In [20]:
# predict on test data
test_preds = model.predict_proba(stest)[:,1]

3706432/3706907 [============================>.] - ETA: 0s

In [21]:
# create submission file
submit = pd.DataFrame({'ID':test.ID, 'click':test_preds})
submit.to_csv('Submission/Keras_subNN1.csv', index=False)

In [23]:
from xgboost import XGBClassifier

/home/anirudh/anaconda/lib/python2.7/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [25]:
model_XGB = XGBClassifier()

In [ ]:
y__pred = model.predict(X_train)